In [42]:
import numpy as np

class Polynom:
    def __init__(self, arg):
        if isinstance(arg, int):
            assert arg >= 0, f"Expected non-negative degree, got {arg}."
            self.deg = arg
            self.coeffs = np.zeros(arg + 1)
        elif isinstance(arg, np.ndarray):
            assert arg.ndim == 1, "Coefficients must be a 1D array."
            self.coeffs = arg.astype(float)
            self.deg = len(self.coeffs) - 1
        else:
            raise TypeError("Polynom constructor expects an int or a 1D numpy array.")

    def __call__(self, x: float):
        result = 0.0
        power = 1.0
        for c in self.coeffs:
            result += c * power
            power *= x
        return result

    def __str__(self):
        return f"<Polynom deg={self.deg}, coeffs={self.coeffs}>"

    def set_coeff(self, id_coeff:int, coeff:float):
        assert id_coeff<=self.deg and id_coeff>=0, ValueError(
            f"Expected a coefficient the polynom has, got {id_coeff}.")
        self.coeffs[id_coeff] = coeff
    
    def get_coeff(self, id_coeff:int):
        assert id_coeff<=self.deg and id_coeff>=0, ValueError(
            f"Expected a coefficient the polynom has, got {id_coeff}.")
        return self.coeffs[id_coeff]

    def set_deg(self, deg:int):
        assert deg>=0, ValueError(
            f"Expected a positive polynom degree, got {deg}.")
        if deg < self.deg:
            self.coeffs = self.coeffs[: deg]
        else:
            self.coeffs = np.append(self.coeffs, np.zeros(deg - self.deg))
        self.deg = deg
    
    def get_deg(self):
        return self.deg

    def derive(self):
        if self.deg == 0:
            self.coeffs = np.array([0.0])
        else:
            deriv = np.empty(self.deg)
            for i in range(1, self.deg+1):
                deriv[i - 1] = i * self.coeffs[i]

            self.deg -= 1
            self.coeffs = deriv
    
    def add(self, p: np.ndarray):
        n = max(self.deg, p.get_deg())
        res = np.zeros(n+1)
        for i in range(self.deg + 1):
            res[i] += self.coeffs[i]
        for i in range(p.get_deg() + 1):
            res[i] += p.coeffs[i]

        last_non_zero = 0
        for i in reversed(range(n + 1)):
            if abs(res[i]) > 1e-14:
                last_non_zero = i
                break
        
        self.deg = last_non_zero
        self.coeffs = res[:last_non_zero + 1]

    def mul_scalar(self, scalar: float):
        self.coeffs *= scalar
        if scalar == 0:
            self.deg = 0

    def mul_X(self):
        res = np.zeros(self.deg + 2)
        for i in range(self.deg + 1):
            res[i + 1] = self.coeffs[i]

        self.deg += 1
        self.coeffs = res

    def copy(self):
        p = Polynom(self.deg)
        p.coeffs = np.copy(self.coeffs)
        return p

def next_assoc_legendre(l:int, m:int, P0:Polynom, P1:Polynom):
    P = Polynom(P1.get_deg()+1)

    a = P1.copy()
    b = P0.copy()

    a.mul_scalar((2*l + 1) / (l - m + 1))
    a.mul_X()

    b.mul_scalar(-(l + m) / (l - m + 1))

    P.add(a)
    P.add(b)
    return P

class AssociatedLegendrePolynomsCalculator:
    """Class for associated Legendre Polynoms for m=2 (weak lensing).
    """
    def __init__(self):
        self.P2 = Polynom(np.array([3, 0, -3]))
        self.P3 = Polynom(np.array([0, 15, 0, -15]))
        self.polynoms = [self.P2, self.P3]
    
    def __str__(self):
        return f"<AssociatedLegendrePolynoms n={len(self.polynoms)}>"
    
    def __call__(self, nb_l:int):
        assert nb_l>=0, ValueError(
            f"Expected l>=0, got {nb_l}.")

        if nb_l<=2:
            return self.polynoms[:l]
        
        P0 = self.P2.copy()
        P1 = self.P3.copy()
        for l in range(3, nb_l+1):
            P_next = next_assoc_legendre(l, 2, P0, P1)
            P0, P1 = P1, P_next
            self.polynoms.append(P_next)
        return self.polynoms


In [58]:
from astropy.io import fits

file_path = 'DES-Y3_xipm_and_KiDS-1000_COSEBIs_2.0_300.0.fits'

with fits.open(file_path) as hdul:
    hdul.info()
    xip_data = hdul['xip'].data
    xim_data = hdul['xim'].data
    desy3_covmat = hdul['COVMAT'].data
    nz_data_des = hdul['nz_source_des'].data
    nz_source_kids = hdul['nz_source_kids'].data

z_vals_des = nz_data_des['Z_MID']
z_vals_kids = nz_source_kids['Z_MID']
n_bin1 = nz_data_des['BIN1']

xi_minus_data = xim_data['VALUE']
xi_minus_ang = xim_data['ANG']
xi_plus_data = xip_data['VALUE']
xi_plus_ang = xip_data['ANG']

xi_minus_data

# polynoms = AssociatedLegendrePolynomsCalculator()
# nb_l=5
# ps = polynoms(nb_l=nb_l)
# for i in range(nb_l):
#     print(f"P(l={i}, m=2) : {ps[i]}")

Filename: DES-Y3_xipm_and_KiDS-1000_COSEBIs_2.0_300.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  nz_source_kids    1 BinTableHDU     53   119R x 8C   [D, D, D, D, D, D, D, D]   
  2  nz_source_des    1 BinTableHDU     47   300R x 7C   [D, D, D, D, D, D, D]   
  3  En            1 BinTableHDU     29   75R x 5C   [D, D, D, D, D]   
  4  xip           1 BinTableHDU     54   200R x 8C   [K, K, K, D, D, D, D, D]   
  5  xim           1 BinTableHDU     54   200R x 8C   [K, K, K, D, D, D, D, D]   
  6  COVMAT        1 ImageHDU        14   (475, 475)   float64   


array([ 5.60464931e-06,  8.15338604e-06,  3.19904575e-06,  4.97486936e-06,
        4.57208726e-06,  5.38683064e-07,  2.48577274e-06,  3.71659489e-07,
        1.65370857e-06,  7.53399116e-07,  1.42202663e-06,  1.20089391e-06,
        5.92986711e-08,  3.20001593e-07, -1.00534252e-08,  3.61402789e-07,
        1.65350783e-07,  1.90954332e-07,  1.26948503e-07,  1.40228307e-07,
        2.39754540e-06,  2.71998712e-06,  1.36507000e-06,  2.84209497e-06,
        3.36919226e-06,  3.29984224e-07,  3.09421196e-06,  3.02651998e-06,
        1.93782347e-06,  2.71377255e-07,  2.12967267e-06,  1.01145534e-06,
        9.44801682e-07,  9.26831517e-07,  5.97527375e-07,  2.74035957e-07,
        4.84752702e-07,  2.94883891e-07,  2.56298851e-07,  1.87298608e-07,
       -3.42187117e-06,  2.41502906e-06,  6.80766965e-06,  1.07674850e-05,
        3.39335635e-06,  1.57199129e-06,  3.49585341e-06,  1.78813687e-06,
        1.55300036e-06,  1.55329573e-06,  9.54772662e-07,  6.10941845e-07,
        7.31139170e-07,  

In [6]:
def Gp(x:float, l:int, m:int=2):
    return - ( (l-m**2) / (1-x**2) + 0.5 * l * (l-1) ) * ps[l](x) + (l+m) * (x / (1-x**2)) * ps[l-1](x)

def Gm(theta:float):
    return m * ( (l-1) * x / (1-x**2) * ps[l](x) - (l+m) * (1 / (1-x**2)) * ps[l-1](x))

###

def _H_LCDM(z, H0, omega_m):
    return H0 * np.sqrt(omega_m * (1+z)**3 + (1 - omega_m))

def _inv_H_LCDM(z, H0, omega_m):
    return 1.0 / _H_LCDM(z, H0, omega_m)

def integral_trapezoid(func, a, b, N, **kwargs):
    h = (b - a) / N
    result = 0.5 * (func(a, **kwargs) + func(b, **kwargs))
    for i in range(1, N):
        result += func(a + i * h, **kwargs)
    result *= h
    return result

def chi(z, H0, omega_m):
    return c * integral_trapezoid(_inv_H_LCDM, 0.0, z, 100, H0=H0, omega_m=omega_m)

def get_z(chi_val):
    return 0

## not njit
def get_n(i, z):
    i_nearest_z = np.argmin(np.abs(z_vals - 1))
    return nz_data_des['BIN' + str(i)][i_nearest_z]

def integrand_w(chi_val, chi_val_2, z_chi_val, n_i_z):
    dz_dchi = 1 ## np.gradient(chi_grid, zgrid)
    return n_i_z * dz_dchi * (chi_val - chi_val_2) / chi_val

def w(i, chi_val, chi_H, H0, omega_m):
    z_chi = get_z(chi=chi_val)
    n_i_z = get_n_i(i=i, z=z_chi)
    integral = integral_trapezoid(integrand_w, chi_val, chi_H, 100, chi_val_2=chi_val, z_chi_val=z_chi, n_i_z=n_i_z)
    return 1.5 * omega_m * (H0/c)**2 * chi_val * (1+z_chi) * integral

def prob(k, z):
    return 0.0

def integrand_ClEE(chi_val, i, j, l, H0, omega_m, chi_H, z_chi_val):
    return w(i, chi_val, chi_H, H0, omega_m) * w(j, chi_val, chi_H, H0, omega_m) / (chi_val**2) * prob(k=(l+0.5)/chi_val, z=z_chi_val)

def ClEE(i:int, j:int, l:int, chi_H):
    return integral_trapezoid(integrand_ClEE, 0, chi_H, 100, i=i, j=j, l=l, H0=H0, omega_m=omega_m, chi_H=chi_H, z_chi_val=z_chi_val)

def ClBB(i:int, j:int, l:int):
    return 0.0

def xip(theta:float, i:int, j:int):
    xi = 0.0
    for l in range(nb_l):
        xi += (2*l + 1)/(2*np.pi*(l+1)**2) * (Gp(np.cos(theta)) + Gm(np.cos(theta))) * (ClEE(i, j, l) + ClBB(i, j, l))
    return xi

def chi2_wl():
    delta_xi = np.empty(len(xi_plus_data))
    for i in range(len(xi_plus_data)):
        bin1 = xip_data['BIN1'][i]
        bin2 = xip_data['BIN2'][i]
        delta_xi[i] = xi_plus_data[i] - xip(xi_plus_ang[i], bin1, bin2)
    return delta_xi @ np.linalg.inv(desy3_covmat) @ delta_xi